In [24]:
import pandas as pd
import numpy as np

In [3]:
###### 1 Min Data - 1 Step Forecasting ######

In [4]:
def CreateData(data_c, shif, train_index):
    data_c.drop("system_time", axis = 1)
    data_c['Next_mid'] = data_c.midpoint.shift(-shif)
    data_c['Price_move'] = (data_c.Next_mid - data_c.midpoint) / data_c.midpoint
    data_c.dropna(axis=0, inplace=True) #drop the last row which is now Nan

    data_c['y'] = 0
    data_c.loc[(data_c['Price_move'] < -0.001), 'y'] = 1
    data_c.loc[(data_c['Price_move'] > 0.001), 'y'] = 2
    data_c.drop(['Next_mid', 'midpoint', 'Price_move', "system_time", "Return"], axis=1, inplace=True)
    Y = data_c.pop("y")
    
    data_std = data_c

    r, x = data_c.shape

    for j in range(x):
        m = data_c.iloc[:train_index,j].mean()
        s = data_c.iloc[:train_index,j].std()
        for i in range(r):
            data_std.iloc[i,j] = (data_c.iloc[i,j] - m) / s
    
    return data_std, Y

In [25]:
def CreateBidAskData(data_c):
    bids = data_c.iloc[:,0:18]
    asks = data_c.iloc[:,78:93]
    data_bid_ask  = pd.concat([bids.reset_index(drop=True), asks], axis=1)
    
    return data_bid_ask
    

In [26]:
def CreateOderFlowData(data_c):
    data_order_flow = data_c.iloc[:,:153]
    
    return(data_order_flow)

In [50]:
3351/12

279.25

In [51]:
3350-280*2

2790

In [52]:
data_1min_std, Y_1min = CreateData(data_1min, 1, 14000)
data_1min = pd.read_csv("data/final/1min/ADA.csv")
data_5min = pd.read_csv("data/final/5min/ADA.csv")

data_1min = pd.read_csv("data/final/1min/ADA.csv")



In [53]:
Y_1min.value_counts()

0    9675
2    3738
1    3631
Name: y, dtype: int64

In [27]:
def CreateData(data_c, shif, train_index):
    data_c.drop("system_time", axis = 1)
    data_c['Next_mid'] = data_c.midpoint.shift(-shif)
    data_c['Price_move'] = (data_c.Next_mid - data_c.midpoint) / data_c.midpoint
    data_c.dropna(axis=0, inplace=True) #drop the last row which is now Nan

    data_c['y'] = 0
    data_c.loc[(data_c['Price_move'] < -0.002), 'y'] = 1
    data_c.loc[(data_c['Price_move'] > 0.002), 'y'] = 2
    data_c.drop(['Next_mid', 'midpoint', 'Price_move', "system_time", "Return"], axis=1, inplace=True)
    Y = data_c.pop("y")
    
    data_std = data_c

    r, x = data_c.shape

    for j in range(x):
        m = data_c.iloc[:train_index,j].mean()
        s = data_c.iloc[:train_index,j].std()
        for i in range(r):
            data_std.iloc[i,j] = (data_c.iloc[i,j] - m) / s
    
    return data_std, Y

In [30]:
data_1min = pd.read_csv("data/final/1min/ADA.csv")
data_5min = pd.read_csv("data/final/5min/ADA.csv")

data_1min_5_std, Y_1_5min = CreateData(data_1min, 5, 14000)
data_5min_std, Y_5min = CreateData(data_5min, 1, 2790)


data_1min_5_std.to_csv("data/final/1min/HF/ADA_std_5.csv", index = False)
Y_1_5min.to_csv("data/final/1min/ADA_Y_5_std.csv", index = False)

OrderFlow1_5 = CreateOderFlowData(data_1min_5_std)
OrderFlow1_5.to_csv("data/final/1min/OrderFlow/ADA_std_5.csv", index = False)

BidAsk1_5 = CreateBidAskData(data_1min_5_std)
BidAsk1_5.to_csv("data/final/1min/BidAsk/ADA_std_5.csv", index = False)

data_5min_std.to_csv("data/final/5min/HF/ADA_std.csv", index = False)
Y_5min.to_csv("data/final/5min/ADA_Y_std.csv", index = False)

OrderFlow5 = CreateOderFlowData(data_5min_std)
OrderFlow5.to_csv("data/final/5min/OrderFlow/ADA_std.csv", index = False)

BidAsk5 = CreateBidAskData(data_5min_std)
BidAsk5.to_csv("data/final/5min/BidAsk/ADA_std.csv", index = False)


In [22]:
Y_5min.value_counts()

0    2126
2     623
1     599
Name: y, dtype: int64

In [57]:
data_1min_std.to_csv("data/final/1min/HF/ADA_std.csv", index = False)
Y_1min.to_csv("data/final/1min/ADA_Y_std.csv", index = False)

OrderFlow1 = CreateOderFlowData(data_1min_std)
OrderFlow1.to_csv("data/final/1min/OrderFlow/ADA_std.csv", index = False)

BidAsk1 = CreateBidAskData(data_1min_std)
BidAsk1.to_csv("data/final/1min/BidAsk/ADA_std.csv", index = False)
